In [8]:
## Query real time data from DingXiangYuan, and keep the latest records every day for each city

## Created on Sat Feb  8 12:41:50 2020
## Author: leebond
#### resource: https://github.com/jianxu305/nCov2019_analysis

import pandas as pd
import matplotlib.pyplot as plt
import utils   # some convenient functions
import pandas as pd
import pickle as pkl
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from googletrans import Translator # package used to translate Chinese into English
translator = Translator()

In [9]:
## Query the latest data
! python dataset.py

2020-02-09 23:06:14Update records successfully to ../data/DXYArea.csv


In [11]:
DXYArea = pd.read_csv('../data/DXYArea.csv')
# select column
DXYArea = DXYArea[['date','country','countryCode','province', 'city', 'confirmed', 'suspected', 'cured', 'dead']]
# clean data 
DXYArea["city"] = DXYArea["city"].str.split('市',expand=True)[1].str[:-1]

DXYArea.tail(2)  

date country countryCode province city  confirmed  suspected  \
7826  2020-02-09      美国          US      NaN  NaN         12          0   
7827  2020-02-09      越南          VN      NaN  NaN         14          0   

      cured  dead  
7826      3     0  
7827      3     0

In [4]:
len(DXYArea) # unique records daily

7828

In [12]:
def isNaN(num):
    return num != num

## Resoruce for Chinese - English Translation
with open('chineseProvince_to_EN.pkl','rb') as f:
    prov_dict = pkl.load(f)

    
with open('chineseCity_to_EN.pkl','rb') as f:
    city_dict = pkl.load(f)    
        
def translate_to_English(data, prov_dict, city_dict):
    """Translate Chinese in dataset to English
    """        
    data['province'] = data['province'].apply(getProvinceTranslation)
    data['city'] = data['city'].apply(getCityTranslation)
    return data
    
def getProvinceTranslation(name):
    if not isNaN(name):
        return prov_dict[name]
    else: 
        return name

def getCityTranslation(name):
    try:
        if not isNaN(name) :
            return city_dict[name]
        else:
            return name
    except KeyError:
        if name != None and len(name)>0:
            print(name + 'cannot be translated, ask Yiran to mannully Translate\n')

In [13]:
daily_frm_DXYArea = translate_to_English(DXYArea, prov_dict, city_dict)

daily_frm_DXYArea.head(3)

date country countryCode        province  city  confirmed  suspected  \
0  2019-12-01      中国          CN             NaN   NaN          1          0   
1  2019-12-01      中国          CN  Hubei Province   NaN          1          0   
2  2019-12-01      中国          CN  Hubei Province   NaN          1          0   

   cured  dead  
0      0     0  
1      0     0  
2      0     0

In [14]:
daily_frm_DXYArea.to_csv ('../data/DXYArea.csv', index = None, header=True)